In [1]:
def variable_summaries(var, name):
    '''Attach a lot of summaries to a Tensor.'''
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)

In [5]:
# Parameters
batch_size = 3
seq_length = 6
rnn_size   = 10
vocab_size = 5
save_dir_GAN = 'models_GAN'
vocab_file = 'simple_vocab.pkl'

In [7]:
# Load Vocabulary
def load_vocab(save_dir_GAN, vocab_file):
    with open(os.path.join(save_dir_GAN, vocab_file)) as f:
            chars, vocab = cPickle.load(f)
    return chars, vocab

In [20]:
# Generator
import tensorflow as tf
import os
import cPickle
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()

def map_to_text(indices, chars):
    text = []
    for index in indices:
        text.append(chars[index])
    return text

def generate():    
    # Vocabulary

    # Initial indices
    batch_indices = tf.constant(3, shape=[batch_size])

    # Embeddings and Logits
    embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
    softmax   = tf.get_variable('softmax', [rnn_size, vocab_size])

    # RNN
    outputs, indices = [], []
    cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
    state = cell.zero_state(batch_size, tf.float32)
    inp = tf.nn.embedding_lookup(embedding, batch_indices)
        
    with tf.variable_scope('rnn'):
        for i in xrange(seq_length):
            indices.append(batch_indices)
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            rnn_out, state = cell(inp, state)
            logits_out = tf.matmul(rnn_out, softmax)
            outputs.append(logits_out)
            batch_indices = tf.squeeze(Categorical(logits_out).sample(n=1))
            inp = tf.nn.embedding_lookup(embedding, batch_indices)
            
    return outputs, indices
            

with tf.Session() as sess: 
    outputs, indices = generate()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    chars, vocab = load_vocab(save_dir_GAN, vocab_file)
    
    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'
    
    for i, batch_indices in enumerate(indices):
        print 'Iteration %d'%i
        print map_to_text(sess.run(batch_indices), chars),'\n'
    
#     for line in indices:
#         print ''.join(line)

Iteration 0
[[-0.18287835 -0.0958939   0.06456404 -0.37236714 -1.10697126]
 [-0.18287835 -0.0958939   0.06456404 -0.37236714 -1.10697126]
 [-0.18287835 -0.0958939   0.06456404 -0.37236714 -1.10697126]] 

Iteration 1
[[-0.3892526   0.65059733 -0.43885985 -0.27126485  0.66572267]
 [ 0.29698253 -0.20217638 -0.46595472  0.5914343   0.6643908 ]
 [-0.53317803  0.45216396 -0.384321   -0.05057469  0.19042294]] 

Iteration 2
[[-0.32650709  0.0749344  -0.23158525 -0.23768994 -1.21404421]
 [ 0.13346942 -0.21788587  0.13201918 -0.28183323 -1.2738893 ]
 [ 0.1022656  -0.10732047 -0.12744111 -0.43471453 -0.88889432]] 

Iteration 3
[[ 0.10360787  0.11802831 -0.46048519  0.77915812  0.61401176]
 [-0.0490162  -0.02536197 -0.20258133 -0.39626852 -1.30988908]
 [-0.42555541  0.03015882 -0.11452354 -0.48365465 -1.53205168]] 

Iteration 4
[[-0.64681828  0.51537561 -0.0290359   0.01668627 -0.45171329]
 [ 0.04502735 -0.12548052 -0.44158348 -0.31045333 -0.44337764]
 [-0.04854381  0.16968703 -0.52634954 -0.17968

In [29]:
tf.reset_default_graph()
logits = tf.constant([1.,2.,3.])
indices = tf.squeeze(Categorical(logits).sample(n=5))

with tf.Session() as sess:
    print 'Indices: ',indices.eval()

Indices:  [2 2 2 2 1]
